In [26]:
import pandas as pd
import numpy as np
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import utils.utils as utils

train = pd.read_csv('../data/mimic-iv-private/triage_stratified_training.csv')
test = pd.read_csv('../data/mimic-iv-private/triage_stratified_2500.csv')

In [27]:
test

,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint
0,18474069,30615360,98.2,71.0,18.0,94.0,92.0,36.0,0,3.0,Dyspnea on exertion
1,10482402,30835613,97.3,68.0,18.0,100.0,131.0,74.0,5,3.0,S/P FALL
2,11668089,30163418,97.6,105.0,22.0,100.0,147.0,76.0,3,2.0,"Chest pain, Dyspnea"
3,17170624,35921297,97.6,110.0,16.0,98.0,99.0,65.0,10,3.0,Abnormal CT
4,17532289,37034357,97.7,85.0,20.0,100.0,134.0,76.0,0,4.0,Rash
...,...,...,...,...,...,...,...,...,...,...,...
2495,12791002,30386504,98.2,63.0,18.0,100.0,149.0,64.0,2,3.0,WOUND EVAL
2496,12938336,34253493,98.2,88.0,18.0,97.0,143.0,63.0,0,2.0,Hemoptysis
2497,13378145,35205258,98.0,114.0,16.0,99.0,131.0,93.0,0,2.0,Seizure
2498,12351481,39111882,100.0,81.0,20.0,92.0,130.0,56.0,0,2.0,Dyspnea


In [28]:
train = train.drop(columns=['subject_id', 'stay_id', 'chiefcomplaint']) 
train

,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity
0,97.9,92.0,20.0,100.0,143.0,82.0,10,3.0
1,97.1,66.0,17.0,100.0,135.0,94.0,9,3.0
2,97.8,67.0,16.0,100.0,111.0,70.0,4,3.0
3,98.6,118.0,18.0,100.0,131.0,73.0,8,3.0
4,98.0,80.0,20.0,98.0,119.0,46.0,13,3.0
...,...,...,...,...,...,...,...,...
386771,98.0,71.0,16.0,100.0,111.0,66.0,7,3.0
386772,97.8,85.0,18.0,97.0,123.0,86.0,6,3.0
386773,98.3,64.0,16.0,98.0,121.0,71.0,7,2.0
386774,98.2,101.0,14.0,99.0,116.0,76.0,8,3.0


In [29]:
test = test.drop(columns=['subject_id', 'stay_id', 'chiefcomplaint'])
test

,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity
0,98.2,71.0,18.0,94.0,92.0,36.0,0,3.0
1,97.3,68.0,18.0,100.0,131.0,74.0,5,3.0
2,97.6,105.0,22.0,100.0,147.0,76.0,3,2.0
3,97.6,110.0,16.0,98.0,99.0,65.0,10,3.0
4,97.7,85.0,20.0,100.0,134.0,76.0,0,4.0
...,...,...,...,...,...,...,...,...
2495,98.2,63.0,18.0,100.0,149.0,64.0,2,3.0
2496,98.2,88.0,18.0,97.0,143.0,63.0,0,2.0
2497,98.0,114.0,16.0,99.0,131.0,93.0,0,2.0
2498,100.0,81.0,20.0,92.0,130.0,56.0,0,2.0


In [30]:
import re

# Define a function to extract numeric values or map text descriptions
def clean_pain(value):
    # Attempt to extract numeric values using regex
    match = re.search(r'\d+(\.\d+)?', str(value))  # Matches numbers like '8', '9.5'
    if match:
        return float(match.group())
    # Map text descriptions to numeric values
    text_mapping = {
        'none': 0, 'mild': 2, 'moderate': 5, 'severe': 8, 'very bad': 10,
        'unbearable': 10, 'uncomfortable': 4, 'not bad': 1
    }
    value_lower = str(value).lower()
    for key, num in text_mapping.items():
        if key in value_lower:
            return num
    # Return NaN for unprocessable values
    return None

# Apply the function to the pain column
train['pain'] = train['pain'].apply(clean_pain)
test['pain'] = test['pain'].apply(clean_pain)
# Fill missing values with the median pain value
train['pain'] = train['pain'].fillna(train['pain'].median())
test['pain'] = test['pain'].fillna(test['pain'].median())

#convert acuity to int
train['acuity'] = train['acuity'].apply(lambda x: int(x))
test['acuity'] = test['acuity'].apply(lambda x: int(x))
# Adjust class labels to start from 0
train['acuity'] = train['acuity'] - 1
test['acuity'] = test['acuity'] - 1


In [31]:
test

,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity
0,98.2,71.0,18.0,94.0,92.0,36.0,0.0,2
1,97.3,68.0,18.0,100.0,131.0,74.0,5.0,2
2,97.6,105.0,22.0,100.0,147.0,76.0,3.0,1
3,97.6,110.0,16.0,98.0,99.0,65.0,10.0,2
4,97.7,85.0,20.0,100.0,134.0,76.0,0.0,3
...,...,...,...,...,...,...,...,...
2495,98.2,63.0,18.0,100.0,149.0,64.0,2.0,2
2496,98.2,88.0,18.0,97.0,143.0,63.0,0.0,1
2497,98.0,114.0,16.0,99.0,131.0,93.0,0.0,1
2498,100.0,81.0,20.0,92.0,130.0,56.0,0.0,1


In [84]:
# Logistic Regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


X_train = train.drop(columns=['acuity'])
y_train = train['acuity']
X_test = test.drop(columns=['acuity'])
y_test = test['acuity']

# Subsets of training data
train_sizes = [0.01, 0.1, 1.0]  # 1%, 10%, 100%
metrics_results = []

# Train and evaluate for each subset
for size in train_sizes:
    if size < 1.0:
        # Subset the training data for 1% or 10%
        X_train_subset, _, y_train_subset, _ = train_test_split(X_train, y_train, train_size=size, random_state=42)
    else:
        # Use the entire training set for 100%
        X_train_subset, y_train_subset = X_train, y_train
    
    # Train logistic regression
    model = LogisticRegression(max_iter=1000, random_state=42, n_jobs=4)
    model.fit(X_train_subset, y_train_subset)
    
    # Make predictions
    y_pred = model.predict(X_test)

    # Store results
    evaluation_metrics = utils.evaluate_predictions(y_pred, y_test, ordinal=True,flexibility=1, by_class=True)

    metrics_results.append({
        "Training Size": f"{int(size * 100)}%",
        "Metrics": evaluation_metrics
    })

# Display results
results_df = pd.DataFrame(metrics_results)
results_df


/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classificatio

,Training Size,Metrics
0,1%,"{'overall': {'accuracy': 0.5664, 'precision': ..."
1,10%,"{'overall': {'accuracy': 0.57, 'precision': 0...."
2,100%,"{'overall': {'accuracy': 0.5616, 'precision': ..."


In [86]:
results_df.loc[0,'Metrics']

{'overall': {'accuracy': 0.5664,
  'precision': 0.5064142127803721,
  'recall': 0.5664,
  'f1_score': 0.47597480192151725,
  'adjusted_accuracy': 0.97,
  'adjusted_precision': 0.9674674852150139,
  'adjusted_recall': 0.97,
  'adjusted_f1': 0.9614351248487119,
  'mae': 0.4636,
  'mse': 0.5236,
  'quadratic_kappa': 0.12403315745293209},
 'by_class': {'0': {'precision': 0.5714285714285714,
   'recall': 0.046511627906976744,
   'f1-score': 0.08602150537634409,
   'support': 86.0},
  '1': {'precision': 0.4980694980694981,
   'recall': 0.15412186379928317,
   'f1-score': 0.2354014598540146,
   'support': 837.0},
  '2': {'precision': 0.5743061772605192,
   'recall': 0.9210337401292176,
   'f1-score': 0.7074717397298043,
   'support': 1393.0},
  '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 177.0},
  '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7.0},
  'accuracy': 0.5664,
  'macro avg': {'precision': 0.3287608493517177,
   'recall': 0.2243334463670954

In [61]:
pd.DataFrame({'yo': np.array(y_pred), 'yo1': 'a'}).groupby('yo').count()

,yo1
yo,
0,519
1,332
2,383
3,618
4,648


In [36]:
pd.DataFrame(y_pred+1).to_csv('lr_pred.csv', index=False)

In [62]:
print(results_df.loc[2,'Metrics'])

{'overall': {'accuracy': 0.2048, 'precision': 0.5185326864809315, 'recall': 0.2048, 'f1_score': 0.2558379230308958, 'adjusted_accuracy': 0.5944, 'adjusted_precision': 0.921639640385146, 'adjusted_recall': 0.5944, 'adjusted_f1': 0.7008622376025149, 'mae': 1.3288, 'mse': 2.6664, 'quadratic_kappa': 0.09429269068929624}, 'by_class': {'0': {'precision': 0.08477842003853564, 'recall': 0.5116279069767442, 'f1-score': 0.14545454545454545, 'support': 86.0}, '1': {'precision': 0.41566265060240964, 'recall': 0.16487455197132617, 'f1-score': 0.23609923011120615, 'support': 837.0}, '2': {'precision': 0.660574412532637, 'recall': 0.18162239770279973, 'f1-score': 0.2849099099099099, 'support': 1393.0}, '3': {'precision': 0.11812297734627832, 'recall': 0.4124293785310734, 'f1-score': 0.18364779874213835, 'support': 177.0}, '4': {'precision': 0.006172839506172839, 'recall': 0.5714285714285714, 'f1-score': 0.012213740458015267, 'support': 7.0}, 'accuracy': 0.2048, 'macro avg': {'precision': 0.2570622600

In [37]:
kate_df = pd.read_csv('../results/Triage-Private-Stratified/Triage-Private-Stratified_CoT_openai-gpt-4o-chat_json_detailed0_2500.csv')

In [43]:
kate_df['lr_pred'] = y_pred+1
kate_df[['Estimated_Acuity','lr_pred','acuity']].to_csv('2preds_1test.csv', index=False)

In [67]:
from xgboost import XGBClassifier

from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score, cohen_kappa_score, 
                             classification_report, mean_absolute_error, mean_squared_error)

# Subsets of training data
train_sizes = [0.01, 0.1, 1.0]  # 1%, 10%, 100%
metrics_results_xgb = []

# Train and evaluate for each subset
for size in train_sizes:
    if size < 1.0:
        # Subset the training data for 1% or 10%
        X_train_subset, _, y_train_subset, _ = train_test_split(X_train, y_train, train_size=size, random_state=42)
    else:
        # Use the entire training set for 100%
        X_train_subset, y_train_subset = X_train, y_train
    
    # Train XGBoost classifier
    model = XGBClassifier(use_label_encoder=False, eval_metric=cohen_kappa_score, random_state=42)
    model.fit(X_train_subset, y_train_subset)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Store results
    evaluation_metrics = utils.evaluate_predictions(y_pred, y_test, ordinal=True,flexibility=1, by_class=True)

    metrics_results_xgb.append({
        "Training Size": f"{int(size * 100)}%",
        "Metrics": evaluation_metrics
    })

# Display results
results_df_xgb = pd.DataFrame(metrics_results_xgb)
results_df_xgb


/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:37:07] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"

,Training Size,Metrics
0,1%,"{'overall': {'accuracy': 0.5648, 'precision': ..."
1,10%,"{'overall': {'accuracy': 0.5884, 'precision': ..."
2,100%,"{'overall': {'accuracy': 0.602, 'precision': 0..."


In [69]:
results_df_xgb.loc[2,'Metrics']

{'overall': {'accuracy': 0.602,
  'precision': 0.5516411100478469,
  'recall': 0.602,
  'f1_score': 0.5497740145090242,
  'adjusted_accuracy': 0.98,
  'adjusted_precision': 0.9774265382757239,
  'adjusted_recall': 0.98,
  'adjusted_f1': 0.9769243508357329,
  'mae': 0.4184,
  'mse': 0.46,
  'quadratic_kappa': 0.31933974771771656},
 'by_class': {'0': {'precision': 0.7454545454545455,
   'recall': 0.47674418604651164,
   'f1-score': 0.5815602836879432,
   'support': 86.0},
  '1': {'precision': 0.56,
   'recall': 0.3010752688172043,
   'f1-score': 0.3916083916083916,
   'support': 837.0},
  '2': {'precision': 0.6075187969924812,
   'recall': 0.8700646087580761,
   'f1-score': 0.7154663518299882,
   'support': 1393.0},
  '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 177.0},
  '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7.0},
  'accuracy': 0.602,
  'macro avg': {'precision': 0.3825946684894054,
   'recall': 0.32957681272435846,
   'f1-score': 0.337

In [73]:
test.drop(columns=['acuity']).values.shape

(2500, 7)

In [74]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Load training and test data
embeddings = np.load('../data/mimic-iv-private/symptom_embeddings.npy', allow_pickle=True)
test_embeddings= np.load('../data/mimic-iv-private/symptom_embeddings_test.npy', allow_pickle=True)

# Combine vitals with embeddings
X_train = np.hstack([train.drop(columns=['acuity']).values, embeddings])  # Combine vitals and embeddings
y_train = train['acuity'].values

X_test = np.hstack([test.drop(columns=['acuity']).values, test_embeddings])  # Replace with inference embeddings if different
y_test = test['acuity'].values

# Subsets of training data
train_sizes = [1.0]  # 1%, 10%, 100%
metrics_results = []

# Train and evaluate for each subset
X_train_subset, y_train_subset = X_train, y_train

# Train an MLP classifier
model = MLPClassifier(
    hidden_layer_sizes=(775, 64),  # Small network
    random_state=42,
    early_stopping=True
)
model.fit(X_train_subset, y_train_subset)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the predictions
evaluation_metrics = classification_report(y_test, y_pred, output_dict=True)

# Store results
metrics_results.append({
    "Training Size": f"{int(size * 100)}%",
    "Metrics": evaluation_metrics
})

# Display results
results_df = pd.DataFrame([
    {"Training Size": result["Training Size"], **result["Metrics"]["accuracy"], **result["Metrics"]}
    for result in metrics_results
])
print(results_df)

/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

TypeError: 'float' object is not a mapping

In [76]:
utils.evaluate_predictions(y_pred,y_test,ordinal=True, by_class=True)

/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

{'overall': {'accuracy': 0.7036,
  'precision': 0.69382771984375,
  'recall': 0.7036,
  'f1_score': 0.6916408420473397,
  'adjusted_accuracy': 0.9916,
  'adjusted_precision': 0.9916944355529995,
  'adjusted_recall': 0.9916,
  'adjusted_f1': 0.991091579591339,
  'mae': 0.3048,
  'mse': 0.3216,
  'quadratic_kappa': 0.5809021487834903},
 'by_class': {'0': {'precision': 0.7894736842105263,
   'recall': 0.3488372093023256,
   'f1-score': 0.4838709677419355,
   'support': 86.0},
  '1': {'precision': 0.6786155747836835,
   'recall': 0.6559139784946236,
   'f1-score': 0.6670716889428918,
   'support': 837.0},
  '2': {'precision': 0.7272727272727273,
   'recall': 0.8155061019382628,
   'f1-score': 0.7688663282571913,
   'support': 1393.0},
  '3': {'precision': 0.4835164835164835,
   'recall': 0.24858757062146894,
   'f1-score': 0.3283582089552239,
   'support': 177.0},
  '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7.0},
  'accuracy': 0.7036,
  'macro avg': {'precision': 0

In [88]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Load training and test data
embeddings = np.load('../data/mimic-iv-private/symptom_embeddings.npy', allow_pickle=True)
test_embeddings = np.load('../data/mimic-iv-private/symptom_embeddings_test.npy', allow_pickle=True)

# Combine vitals with embeddings
X_train = np.hstack([train.drop(columns=['acuity']).values, embeddings])  # Combine vitals and embeddings
y_train = train['acuity'].values

X_test = np.hstack([test.drop(columns=['acuity']).values, test_embeddings])  # Replace with inference embeddings if different
y_test = test['acuity'].values

# Subsets of training data
train_sizes = [0.01, 0.1]  # 1%, 10%, 100%
metrics_results = []

# Train and evaluate for each subset
for size in train_sizes:
    # Create a subset of the training data
    subset_size = int(len(X_train) * size)
    X_train_subset, _, y_train_subset, _ = train_test_split(
        X_train, y_train, train_size=subset_size, random_state=42, stratify=y_train
    )

    # Train an MLP classifier
    model = MLPClassifier(
        hidden_layer_sizes=(775, 64),  # Small network
        random_state=42,
        early_stopping=True
    )
    model.fit(X_train_subset, y_train_subset)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the predictions
    evaluation_metrics = utils.evaluate_predictions(y_pred,y_test,ordinal=True, by_class=True)

    # Store results
    metrics_results.append(evaluation_metrics)

# Display results
results_df = pd.DataFrame(metrics_results)
print(results_df)

/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

                                             overall  \
0  {'accuracy': 0.6396, 'precision': 0.6202773153...   
1  {'accuracy': 0.6776, 'precision': 0.6687991886...   

                                            by_class  
0  {'0': {'precision': 0.42424242424242425, 'reca...  
1  {'0': {'precision': 0.53125, 'recall': 0.19767...  


In [91]:
results_df.loc[1]['overall']

{'accuracy': 0.6776,
 'precision': 0.6687991886557905,
 'recall': 0.6776,
 'f1_score': 0.6661262548531941,
 'adjusted_accuracy': 0.9864,
 'adjusted_precision': 0.9861556950476779,
 'adjusted_recall': 0.9864,
 'adjusted_f1': 0.985951754713654,
 'mae': 0.336,
 'mse': 0.3632,
 'quadratic_kappa': 0.5416948648827627}

In [92]:
results_df.loc[1]['by_class']

{'0': {'precision': 0.53125,
  'recall': 0.19767441860465115,
  'f1-score': 0.288135593220339,
  'support': 86.0},
 '1': {'precision': 0.6739446870451238,
  'recall': 0.5531660692951016,
  'f1-score': 0.6076115485564304,
  'support': 837.0},
 '2': {'precision': 0.7041383570105003,
  'recall': 0.8183776022972002,
  'f1-score': 0.7569721115537849,
  'support': 1393.0},
 '3': {'precision': 0.45962732919254656,
  'recall': 0.4180790960451977,
  'f1-score': 0.4378698224852071,
  'support': 177.0},
 '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7.0},
 'accuracy': 0.6776,
 'macro avg': {'precision': 0.47379207464963413,
  'recall': 0.39745943724843014,
  'f1-score': 0.4181178151631523,
  'support': 2500.0},
 'weighted avg': {'precision': 0.6687991886557905,
  'recall': 0.6776,
  'f1-score': 0.6661262548531941,
  'support': 2500.0}}